In [22]:
import requests
import xmltodict
from IPython.display import JSON
import pandas as pd

In [ ]:
req = requests.get(f"https://vocabularies.coar-repositories.org/resource_types/1.1/resource_types_for_dspace_en.xml")
data=xmltodict.parse(req.text)

In [16]:
JSON(data)

<IPython.core.display.JSON object>

In [17]:
def extract_nodes(data, result=None):
    if result is None:
        result = []

    if isinstance(data, dict):
        if '@id' in data and '@label' in data:
            result.append({
                '@id': data['@id'],
                '@label': data['@label'],
                'hasNote': data.get('hasNote', '')
            })

        if 'isComposedBy' in data and 'node' in data['isComposedBy']:
            nodes = data['isComposedBy']['node']
            if isinstance(nodes, dict):
                extract_nodes(nodes, result)
            elif isinstance(nodes, list):
                for node in nodes:
                    extract_nodes(node, result)

    return result

In [27]:
#https://vocabularies.coar-repositories.org/resource_types/1.1/
df=pd.DataFrame()
versions=["1.1","2.0","3.1","3.2"]
for i in versions:
    req = requests.get(f"https://vocabularies.coar-repositories.org/resource_types/{i}/resource_types_for_dspace_en.xml")
    data=xmltodict.parse(req.text)
    values=extract_nodes(data["node"])
    df = pd.concat([df, pd.DataFrame(values)], ignore_index=True)
df

,@id,@label,hasNote
0,http://purl.org/coar/resource_type,Resource Types,
1,c_12cc,cartographic material,Any material representing the whole or part of...
2,c_12cd,map,Defined as a representation normally to scale ...
3,c_ddb1,dataset,A collection of related facts and data encoded...
4,c_c513,image,"A visual representation other than text, inclu..."
...,...,...,...
323,c_bdcc,master thesis,A thesis reporting a research project undertak...
324,6NC7-GK9S,transcription,A written record of words spoken in court proc...
325,c_8042,working paper,A working or discussion paper circulated publi...
326,H6QP-SC1X,trademark,A sign used to distinguish the goods or servic...


In [28]:
df_unique = df.drop_duplicates(subset=['@id'], keep='first')
df_unique

,@id,@label,hasNote
0,http://purl.org/coar/resource_type,Resource Types,
1,c_12cc,cartographic material,Any material representing the whole or part of...
2,c_12cd,map,Defined as a representation normally to scale ...
3,c_ddb1,dataset,A collection of related facts and data encoded...
4,c_c513,image,"A visual representation other than text, inclu..."
...,...,...,...
255,GSZA-Y7V7,knowledge organization system,All types of schemes for organizing informatio...
265,S7R1-K5P0,physical sample,"Physical samples include biological specimens,..."
266,8KJG-QS0Y,research instrument,"A device, tool or apparatus used to obtain, me..."
297,43KC-T6DC,magazine article,An Item appearing in a magazine (a general int...


In [31]:
df_unique.to_excel("coar_types.xlsx")